In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./datasets/DfCleaned.csv')
df.head()

,Unnamed: 0,resp_text,clarity,Class,tokens,lemma,ngram2,ngram3
0,0,"Prezada Senhora, Informa-se que o DNIT é uma ...",c1,0,"['prezada', 'senhora', 'informase', 'que', 'o'...",prezar senhor informase que o dnit ser um auta...,"[('prezar', 'senhor'), ('senhor', 'informase')...","[('prezar', 'senhor', 'informase'), ('senhor',..."
1,1,"Prezada, Os dados se referem somente aos na...",c1,0,"['prezada', 'os', 'dados', 'se', 'referem', 's...",prezar o dar se referir somente a o nascido vi...,"[('prezar', 'o'), ('o', 'dar'), ('dar', 'se'),...","[('prezar', 'o', 'dar'), ('o', 'dar', 'se'), (..."
2,2,"Prezado Usuário, Inicialmente agradecemos o s...",c1,0,"['prezado', 'usuário', 'inicialmente', 'agrade...",prezar usuário inicialmente agradecer o seu co...,"[('prezar', 'usuário'), ('usuário', 'inicialme...","[('prezar', 'usuário', 'inicialmente'), ('usuá..."
3,3,"Prezada, Referimo-nos a sua demanda, regist...",c1,0,"['prezada', 'referimonos', 'a', 'sua', 'demand...",prezar referimono o seu demanda registrar em o...,"[('prezar', 'referimono'), ('referimono', 'o')...","[('prezar', 'referimono', 'o'), ('referimono',..."
4,4,"Prezada, Segue anexa a resposta da área com...",c1,0,"['prezada', 'segue', 'anexa', 'a', 'resposta',...",prezar seguir anexo o resposta de o área compe...,"[('prezar', 'seguir'), ('seguir', 'anexo'), ('...","[('prezar', 'seguir', 'anexo'), ('seguir', 'an..."


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

2023-11-10 21:30:29.124963: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 21:30:29.125016: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 21:30:29.125065: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
import tensorflow_hub as hub
import tensorflow_text as text

In [6]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [7]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

### Bert-Evaluate

In [8]:
x_train = df['tokens']
y_train = df['Class']

In [9]:
# x_train = np.array(df['pos']).reshape(-1,1)
# x_train

In [10]:
from sklearn.model_selection import KFold

In [11]:
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True)

In [12]:

results = []

# Iterate over the folds
for train_index, test_index in kfold.split(x_train):
    x_train_fold, x_val_fold = x_train[train_index], x_train[test_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[test_index]

    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)

    l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
    l = tf.keras.layers.Dense(32, activation='relu', name="first")(l)
    l = tf.keras.layers.Dense(64, activation='relu', name="second")(l)
    l = tf.keras.layers.Dense(3, activation='softmax', name="output")(l)

    model = tf.keras.Model(inputs=[text_input], outputs=[l])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model on the current fold
    model.fit(x_train_fold, y_train_fold, epochs=5)  # Adjust the number of epochs as needed

    # Evaluate the model on the validation data for this fold
    _, accuracy = model.evaluate(x_val_fold, y_val_fold)
    print(f"result -> {accuracy}")
    results.append(accuracy)


TypeError: Exception encountered when calling layer "keras_layer_1" (type KerasLayer).

in user code:

    File "/home/chocomenta/anaconda3/lib/python3.11/site-packages/tensorflow_hub/keras_layer.py", line 234, in call  *
        result = f()

    TypeError: Binding inputs to tf.function failed due to `Expected string, but got Tensor("Placeholder:0", shape=(None, 128), dtype=int32) of type 'SymbolicTensor'.`. Received args: ({'input_type_ids': <tf.Tensor 'Placeholder_1:0' shape=(None, 128) dtype=int32>, 'input_mask': <tf.Tensor 'Placeholder:0' shape=(None, 128) dtype=int32>, 'input_word_ids': <tf.Tensor 'Placeholder_2:0' shape=(None, 128) dtype=int32>},) and kwargs: {} for signature: (inputs: TensorSpec(shape=<unknown>, dtype=tf.string, name=None)).


Call arguments received by layer "keras_layer_1" (type KerasLayer):
  • inputs={'input_type_ids': 'tf.Tensor(shape=(None, 128), dtype=int32)', 'input_mask': 'tf.Tensor(shape=(None, 128), dtype=int32)', 'input_word_ids': 'tf.Tensor(shape=(

In [ ]:

def modelBERT():

    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)

    l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
    l = tf.keras.layers.Dense(32, activation='relu', name="first")(l)
    l = tf.keras.layers.Dense(64, activation='relu', name="second")(l)
    l = tf.keras.layers.Dense(3, activation='softmax', name="output")(l)

    model = tf.keras.Model(inputs=[text_input], outputs=[l])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:

# Calculate the mean of the results
mean_accuracy = np.mean(results)
print("Mean cross-validated accuracy:", mean_accuracy)

In [ ]:
from scikeras.wrappers import KerasClassifier

In [ ]:
clf = KerasClassifier(build_fn=modelBERT, epochs=2, batch_size=32, verbose=1)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
result = cross_val_score(clf, x_train, y_train, scoring='accuracy', cv=3)
result

### test